#### Coutresy to convert XML to JSON https://stackoverflow.com/questions/191536/converting-xml-to-json-using-python 
Made small modifications for our purpose
Remove certain first few lines from XML files and run the code by changing the XML filename

In [1]:
from xml.dom import minidom
import simplejson as json
def parse_element(element):
    dict_data = dict()
    if element.nodeType == element.TEXT_NODE:
        if element.data != " ":
            dict_data['data'] = element.data
    if element.nodeType not in [element.TEXT_NODE, element.DOCUMENT_NODE, 
                                element.DOCUMENT_TYPE_NODE]:
        for item in element.attributes.items():
            dict_data[item[0]] = item[1]
    if element.nodeType not in [element.TEXT_NODE, element.DOCUMENT_TYPE_NODE]:
        for child in element.childNodes:
            child_name, child_dict = parse_element(child)
            if child_name in dict_data:
                try:
                    dict_data[child_name].append(child_dict)
                except AttributeError:
                    dict_data[child_name] = [dict_data[child_name], child_dict]
            else:
                dict_data[child_name] = child_dict 
    return element.nodeName, dict_data

if __name__ == '__main__':
    dom = minidom.parse('sample1.xml')
    print type(dom)
    f = open('data12.json', 'w')
    f.write(json.dumps(parse_element(dom), sort_keys=True, indent=4))
    f.close()

<type 'instance'>


In [2]:
import pandas as pd

data = pd.read_json('data12.json')
df3 = pd.io.json.json_normalize(data[0][1]['xml']['Thread'])
df4 = pd.concat([pd.DataFrame(pd.io.json.json_normalize(x)) for x in df3['RelComment']],ignore_index=True)
df4['THREAD_SEQUENCE'] = [ '_'.join(i.split('_')[:2]) for i in df4['RELC_ID']]
df5 = pd.concat([df3.set_index('THREAD_SEQUENCE'),df4.set_index('THREAD_SEQUENCE')], axis=1, join='inner').reset_index()

[ df5.drop(i, axis=1, inplace=True) for i in ['RelQuestion.#text' , 'RelComment', 'RelCBody.#text.data' ,
                                              'RelQuestion.RelQBody.#text.data' , 'RelQuestion.RelQBody.#text.data', '#text'] ]
#df5[ df5['THREAD_SEQUENCE'] == 'Q268_R16' ]
df5.describe()

,THREAD_SEQUENCE,RelQuestion.RELQ_CATEGORY,RelQuestion.RELQ_DATE,RelQuestion.RELQ_ID,RelQuestion.RELQ_USERID,RelQuestion.RELQ_USERNAME,RelQuestion.RelQClean.#text.data,RelQuestion.RelQSubject.#text.data,RELC_DATE,RELC_ID,RELC_RELEVANCE2RELQ,RELC_USERID,RELC_USERNAME,RelCClean.#text.data
count,2440,2440,2440,2440,2440,2440,2440,2440,2440,2440,2440,2440,2440,2440
unique,244,21,244,244,196,196,244,243,2434,2440,3,914,914,2401
top,Q295_R63,Qatar Living Lounge,2013-02-24 09:50:09,Q295_R63,U2,anonymous,Do you know any backbiters in your office? // ...,QP Offer,2011-07-28 14:25:18,Q269_R7_C7,Bad,U2,anonymous,very good but i thing expensive
freq,10,860,10,10,310,310,10,20,3,1,1209,374,374,10


In [34]:
from xml.dom import minidom
import simplejson as json
def parse_element(element):
    dict_data = dict()
    if element.nodeType == element.TEXT_NODE:
        if element.data != " ":
            dict_data['data'] = element.data
    if element.nodeType not in [element.TEXT_NODE, element.DOCUMENT_NODE, 
                                element.DOCUMENT_TYPE_NODE]:
        for item in element.attributes.items():
            dict_data[item[0]] = item[1]
    if element.nodeType not in [element.TEXT_NODE, element.DOCUMENT_TYPE_NODE]:
        for child in element.childNodes:
            child_name, child_dict = parse_element(child)
            if child_name in dict_data:
                try:
                    dict_data[child_name].append(child_dict)
                except AttributeError:
                    dict_data[child_name] = [dict_data[child_name], child_dict]
            else:
                dict_data[child_name] = child_dict 
    return element.nodeName, dict_data

if __name__ == '__main__':
    dom = minidom.parse('cqa_task3.xml')
    print type(dom)
    f = open('cqa_task3.json', 'w')
    f.write(json.dumps(parse_element(dom), sort_keys=True, indent=4))
    f.close()

<type 'instance'>


In [37]:
import pandas as pd

data1 = pd.read_json('cqa_task3.json')
data1.head()
df31 = pd.io.json.json_normalize(data1[0][1]['xml'][1]['Thread'])
# df31['Thread']# df31.apply(pd.io.json.json_normalize)
# df41 = pd.concat([pd.DataFrame(pd.io.json.json_normalize(x)) for x in df31,ignore_index=True)
df31.head()
df41 = pd.concat([pd.DataFrame(pd.io.json.json_normalize(x)) for x in df31['RelComment']],ignore_index=True)
df41['THREAD_SEQUENCE'] = [ '_'.join(i.split('_')[:2]) for i in df41['RELC_ID']]
df51 = pd.concat([df31.set_index('THREAD_SEQUENCE'),df41.set_index('THREAD_SEQUENCE')], axis=1, join='inner').reset_index()
df51.head()
[ df51.drop(i, axis=1, inplace=True) for i in ['RelQuestion.#text' , 'RelComment', 'RelCBody.#text.data' ,
                                              'RelQuestion.RelQBody.#text.data' , 'RelQuestion.RelQBody.#text.data', '#text'] ]
df51[ df51['RelQuestion.RelQSubject.#text.data'] == 'Best Bank.' ]
#df51.describe()

,THREAD_SEQUENCE,RelQuestion.RELQ_CATEGORY,RelQuestion.RELQ_DATE,RelQuestion.RELQ_ID,RelQuestion.RELQ_USERID,RelQuestion.RELQ_USERNAME,RelQuestion.RelQClean.#text.data,RelQuestion.RelQSubject.#text.data,RELC_DATE,RELC_ID,RELC_RELEVANCE2RELQ,RELC_USERID,RELC_USERNAME,RelCClean.#text.data
0,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-07-31 06:46:39,Q268_R16_C1,Bad,U65,Molten Metal,banks are using us ... Talk to those who had t...
1,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-07-31 08:10:53,Q268_R16_C2,Bad,U956,Rip Cord,In Qatar that is like saying which is the best...
2,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-07-31 10:37:03,Q268_R16_C3,Bad,U5152,arman1arzoo,I'm surprised to see such feedbacks on Qatar b...
3,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-08-01 11:38:21,Q268_R16_C4,Good,U5153,westernindoha,Well Arman; nothing is wrong here with banks; ...
4,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-08-01 11:45:49,Q268_R16_C5,Good,U492,happygolucky,With QNB for last 4 years plus...no issues...g...
5,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-08-02 06:38:26,Q268_R16_C6,PotentiallyUseful,U5151,shehabi,WesternInDoha; that's the information that I a...
6,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-08-02 06:46:37,Q268_R16_C7,PotentiallyUseful,U5151,shehabi,MoltenMetal; it depend how you are looking on ...
7,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-08-02 06:51:51,Q268_R16_C8,Bad,U5151,shehabi,That's new way of description for the Bank's.....
8,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-08-02 07:10:40,Q268_R16_C9,Bad,U5151,shehabi,I don't know if it is the high competition? or...
9,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-08-02 12:55:52,Q268_R16_C10,Good,U5153,westernindoha,Sheabi; the reason behind it; is that most peo...


In [1]:
from xml.dom import minidom
import simplejson as json
def parse_element(element):
    dict_data = dict()
    if element.nodeType == element.TEXT_NODE:
        if element.data != " ":
            dict_data['data'] = element.data
    if element.nodeType not in [element.TEXT_NODE, element.DOCUMENT_NODE, 
                                element.DOCUMENT_TYPE_NODE]:
        for item in element.attributes.items():
            dict_data[item[0]] = item[1]
    if element.nodeType not in [element.TEXT_NODE, element.DOCUMENT_TYPE_NODE]:
        for child in element.childNodes:
            child_name, child_dict = parse_element(child)
            if child_name in dict_data:
                try:
                    dict_data[child_name].append(child_dict)
                except AttributeError:
                    dict_data[child_name] = [dict_data[child_name], child_dict]
            else:
                dict_data[child_name] = child_dict 
    return element.nodeName, dict_data

if __name__ == '__main__':
    dom = minidom.parse('Q2Q.xml')
    print type(dom)
    f = open('Q2Q.json', 'w')
    f.write(json.dumps(parse_element(dom), sort_keys=True, indent=4))
    f.close()

<type 'instance'>


In [20]:
import pandas as pd

data111 = pd.read_json('Q2Q.json')
data111.head()
print data111[0][1]['xml'].keys()
df3111 = pd.io.json.json_normalize(data111[0][1]['xml']['OrgQuestion'])
df3111.head()
df4111 = pd.concat([pd.DataFrame(pd.io.json.json_normalize(x)) for x in df3111['Thread.RelComment']],ignore_index=True)
df4111['THREAD_SEQUENCE'] = [ '_'.join(i.split('_')[:2]) for i in df4111['RELC_ID']]
print df4111.count()
df4111.head()

[u'OrgQuestion', u'#text', u'version']
#text                   5000
RELC_DATE               5000
RELC_ID                 5000
RELC_RELEVANCE2ORGQ     5000
RELC_RELEVANCE2RELQ     5000
RELC_USERID             5000
RELC_USERNAME           5000
RelCBody.#text.data     5000
RelCClean.#text.data    5000
THREAD_SEQUENCE         5000
dtype: int64


,#text,RELC_DATE,RELC_ID,RELC_RELEVANCE2ORGQ,RELC_RELEVANCE2RELQ,RELC_USERID,RELC_USERNAME,RelCBody.#text.data,RelCClean.#text.data,THREAD_SEQUENCE
0,"[{u'data': u' '}, {u'data': u' ...",2013-05-03 07:23:20,Q268_R4_C1,Good,Good,U594,Dilgeer,Commercial bank/IBQ,Commercial bank/IBQ,Q268_R4
1,"[{u'data': u' '}, {u'data': u' ...",2013-05-03 12:58:13,Q268_R4_C2,Good,Good,U979,Speedysid,The best bank in Qatar for you would be the on...,The best bank in Qatar for you would be the on...,Q268_R4
2,"[{u'data': u' '}, {u'data': u' ...",2013-05-03 12:58:25,Q268_R4_C3,PotentiallyUseful,PotentiallyUseful,U979,Speedysid,- Credit / Debit card facilities- Customer Car...,- Credit / Debit card facilities - Customer Ca...,Q268_R4
3,"[{u'data': u' '}, {u'data': u' ...",2013-05-03 22:25:47,Q268_R4_C4,Good,Good,U4883,puru1600,Commercial Bank,Commercial Bank,Q268_R4
4,"[{u'data': u' '}, {u'data': u' ...",2013-05-04 07:04:50,Q268_R4_C5,PotentiallyUseful,Good,U4884,usmi,Any bank other than Doha Bank ;),Any bank other than Doha Bank ;),Q268_R4


In [21]:
print df3111.count()
df3111.head()

#text                                                  500
ORGQ_ID                                                500
OrgQBody.#text.data                                    500
OrgQClean.#text.data                                   500
OrgQSubject.#text.data                                 500
Thread.#text                                           500
Thread.RelComment                                      500
Thread.RelQuestion.#text                               500
Thread.RelQuestion.RELQ_CATEGORY                       500
Thread.RelQuestion.RELQ_DATE                           500
Thread.RelQuestion.RELQ_ID                             500
Thread.RelQuestion.RELQ_RANKING_ORDER                  500
Thread.RelQuestion.RELQ_RELEVANCE2ORGQ                 500
Thread.RelQuestion.RELQ_USERID                         500
Thread.RelQuestion.RELQ_USERNAME                       500
Thread.RelQuestion.RelQBody.#text.data                 500
Thread.RelQuestion.RelQClean.#text.data                5

,#text,ORGQ_ID,OrgQBody.#text.data,OrgQClean.#text.data,OrgQSubject.#text.data,Thread.#text,Thread.RelComment,Thread.RelQuestion.#text,Thread.RelQuestion.RELQ_CATEGORY,Thread.RelQuestion.RELQ_DATE,Thread.RelQuestion.RELQ_ID,Thread.RelQuestion.RELQ_RANKING_ORDER,Thread.RelQuestion.RELQ_RELEVANCE2ORGQ,Thread.RelQuestion.RELQ_USERID,Thread.RelQuestion.RELQ_USERNAME,Thread.RelQuestion.RelQBody.#text.data,Thread.RelQuestion.RelQClean.#text.data,Thread.RelQuestion.RelQSubject.#text.data,Thread.SubtaskA_Skip_Because_Same_As_RelQuestion_ID,Thread.THREAD_SEQUENCE
0,"[{u'data': u' '}, {u'data': u' '}, {u'...",Q268,Which is a good bank as per your experience in...,Good Bank // Which is a good bank as per your ...,Good Bank,"[{u'data': u' '}, {u'data': u' '},...","[{u'RELC_RELEVANCE2ORGQ': u'Good', u'#text': [...","[{u'data': u' '}, {u'data': u' ...",Advice and Help,2013-05-02 19:43:00,Q268_R4,4,PerfectMatch,U4882,ankukuma,"Hi Guys,I need to open a new bank accoount. Wh...",Best Bank // Hi Guys; I need to open a new ban...,Best Bank,Q246_R15,Q268_R4
1,"[{u'data': u' '}, {u'data': u' '}, {u'...",Q268,Which is a good bank as per your experience in...,Good Bank // Which is a good bank as per your ...,Good Bank,"[{u'data': u' '}, {u'data': u' '},...","[{u'RELC_RELEVANCE2ORGQ': u'Good', u'#text': [...","[{u'data': u' '}, {u'data': u' ...",Advice and Help,2013-12-31 10:19:32,Q268_R5,5,PerfectMatch,U4880,9chimera11,&lt;p&gt;Seems like all the banks need the sal...,What is the best bank to open an account? // S...,What is the best bank to open an account?,Q246_R13,Q268_R5
2,"[{u'data': u' '}, {u'data': u' '}, {u'...",Q268,Which is a good bank as per your experience in...,Good Bank // Which is a good bank as per your ...,Good Bank,"[{u'data': u' '}, {u'data': u' '},...",[{u'RELC_RELEVANCE2ORGQ': u'PotentiallyUseful'...,"[{u'data': u' '}, {u'data': u' ...",Advice and Help,2015-01-02 03:43:21,Q268_R10,10,PerfectMatch,U4733,Caramellatte,&lt;p&gt;Hi&lt;/p&gt;&lt;p&gt;Does anyone have...,Which Bank to use in Qatar? // Hi Does anyone ...,Which Bank to use in Qatar?,Q229_R13,Q268_R10
3,"[{u'data': u' '}, {u'data': u' '}, {u'...",Q268,Which is a good bank as per your experience in...,Good Bank // Which is a good bank as per your ...,Good Bank,"[{u'data': u' '}, {u'data': u' '},...",[{u'RELC_RELEVANCE2ORGQ': u'PotentiallyUseful'...,"[{u'data': u' '}, {u'data': u' ...",Moving to Qatar,2007-06-27 15:38:30,Q268_R13,13,PerfectMatch,U4909,msmay,"Hi everybody,I need to open a bank account as ...",Which is the best bank around?? // Hi everybod...,Which is the best bank around??,Q246_R76,Q268_R13
4,"[{u'data': u' '}, {u'data': u' '}, {u'...",Q268,Which is a good bank as per your experience in...,Good Bank // Which is a good bank as per your ...,Good Bank,"[{u'data': u' '}, {u'data': u' '},...",[{u'RELC_RELEVANCE2ORGQ': u'PotentiallyUseful'...,"[{u'data': u' '}, {u'data': u' ...",Qatar Living Lounge,2012-02-04 10:43:36,Q268_R14,14,Relevant,U2,anonymous,I would like to apply for a credit card that g...,Best Credit Card in Doha // I would like to ap...,Best Credit Card in Doha,Q51_R41,Q268_R14


In [43]:
df5111 = pd.concat([df3111.set_index('Thread.RelQuestion.RELQ_ID'),df4111.set_index('THREAD_SEQUENCE')], axis=1, join='inner').reset_index()
df5111.columns

remove_index = [ '#text', 'OrgQBody.#text.data' ,'Thread.#text', 'Thread.RelQuestion.#text', 'Thread.RelQuestion.RelQBody.#text.data' 
               ,'Thread.SubtaskA_Skip_Because_Same_As_RelQuestion_ID' , 'index' , 'RelCBody.#text.data' , 'Thread.RelComment']

[ df5111.drop(i, axis=1, inplace=True) for i in remove_index ] 
df5111[ df5111['ORGQ_ID'] == 'Q268' ].columns

Index([                                  u'ORGQ_ID',
                            u'OrgQClean.#text.data',
                          u'OrgQSubject.#text.data',
                u'Thread.RelQuestion.RELQ_CATEGORY',
                    u'Thread.RelQuestion.RELQ_DATE',
           u'Thread.RelQuestion.RELQ_RANKING_ORDER',
          u'Thread.RelQuestion.RELQ_RELEVANCE2ORGQ',
                  u'Thread.RelQuestion.RELQ_USERID',
                u'Thread.RelQuestion.RELQ_USERNAME',
         u'Thread.RelQuestion.RelQClean.#text.data',
       u'Thread.RelQuestion.RelQSubject.#text.data',
                          u'Thread.THREAD_SEQUENCE',
                                       u'RELC_DATE',
                                         u'RELC_ID',
                             u'RELC_RELEVANCE2ORGQ',
                             u'RELC_RELEVANCE2RELQ',
                                     u'RELC_USERID',
                                   u'RELC_USERNAME',
                            u'RelCClean.#text.

In [44]:
df5111.head()

,ORGQ_ID,OrgQClean.#text.data,OrgQSubject.#text.data,Thread.RelQuestion.RELQ_CATEGORY,Thread.RelQuestion.RELQ_DATE,Thread.RelQuestion.RELQ_RANKING_ORDER,Thread.RelQuestion.RELQ_RELEVANCE2ORGQ,Thread.RelQuestion.RELQ_USERID,Thread.RelQuestion.RELQ_USERNAME,Thread.RelQuestion.RelQClean.#text.data,Thread.RelQuestion.RelQSubject.#text.data,Thread.THREAD_SEQUENCE,RELC_DATE,RELC_ID,RELC_RELEVANCE2ORGQ,RELC_RELEVANCE2RELQ,RELC_USERID,RELC_USERNAME,RelCClean.#text.data
0,Q268,Good Bank // Which is a good bank as per your ...,Good Bank,Advice and Help,2013-05-02 19:43:00,4,PerfectMatch,U4882,ankukuma,Best Bank // Hi Guys; I need to open a new ban...,Best Bank,Q268_R4,2013-05-03 07:23:20,Q268_R4_C1,Good,Good,U594,Dilgeer,Commercial bank/IBQ
1,Q268,Good Bank // Which is a good bank as per your ...,Good Bank,Advice and Help,2013-05-02 19:43:00,4,PerfectMatch,U4882,ankukuma,Best Bank // Hi Guys; I need to open a new ban...,Best Bank,Q268_R4,2013-05-03 12:58:13,Q268_R4_C2,Good,Good,U979,Speedysid,The best bank in Qatar for you would be the on...
2,Q268,Good Bank // Which is a good bank as per your ...,Good Bank,Advice and Help,2013-05-02 19:43:00,4,PerfectMatch,U4882,ankukuma,Best Bank // Hi Guys; I need to open a new ban...,Best Bank,Q268_R4,2013-05-03 12:58:25,Q268_R4_C3,PotentiallyUseful,PotentiallyUseful,U979,Speedysid,- Credit / Debit card facilities - Customer Ca...
3,Q268,Good Bank // Which is a good bank as per your ...,Good Bank,Advice and Help,2013-05-02 19:43:00,4,PerfectMatch,U4882,ankukuma,Best Bank // Hi Guys; I need to open a new ban...,Best Bank,Q268_R4,2013-05-03 22:25:47,Q268_R4_C4,Good,Good,U4883,puru1600,Commercial Bank
4,Q268,Good Bank // Which is a good bank as per your ...,Good Bank,Advice and Help,2013-05-02 19:43:00,4,PerfectMatch,U4882,ankukuma,Best Bank // Hi Guys; I need to open a new ban...,Best Bank,Q268_R4,2013-05-04 07:04:50,Q268_R4_C5,PotentiallyUseful,Good,U4884,usmi,Any bank other than Doha Bank ;)
